In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.vision import *
from fastai.callbacks import SaveModelCallback, EarlyStoppingCallback
from fastprogress import progress_bar
import numpy as np
import pandas as pd

import torch
from  torch import nn

from henrys_model_utils import MSELossFlat, Reshape

In [3]:
images_path = '../data/resized_images'
labels_path = '../data/resized_labels.csv'
target_size = 320

In [4]:
y = pd.read_csv(labels_path, index_col='filename')

In [5]:
def get_y_func(x):
    filename = str(x).split('/')[-1]
    coord_list = []
    coords = list(y.loc[filename])
    for i in range(len(coords)//2):
        coord_list.append([coords[i*2+1],coords[i*2]])
    return tensor(coord_list)

In [6]:
transforms = get_transforms(
    do_flip=False,
    max_rotate=45, 
    max_zoom=1.5, 
    max_lighting=0.5, 
    max_warp=0.4, 
    p_affine=1., 
    p_lighting=1.
) 

In [7]:
data = (
    PointsItemList
        .from_folder(images_path)
        .split_by_rand_pct(0.15)
        .label_from_func(get_y_func)
        .transform(
            transforms,
            size=320,
            tfm_y=True, 
            remove_out=False, 
            padding_mode='reflection', 
            resize_method=ResizeMethod.PAD
        )
        .databunch()
        .normalize(imagenet_stats)
)

In [8]:
# data.show_batch(3, figsize=(6,6))

In [9]:
# class Reshape(nn.Module):
#     def __init__(self, *args):
#         super(Reshape, self).__init__()
#         self.shape = args

#     def forward(self, x):
#         return x.view(self.shape)
    
# class MSELossFlat(nn.MSELoss):
#     def forward(self, input, target):
#         return super().forward(input.view(-1), target.view(-1))

In [10]:
head_reg = nn.Sequential(
    Flatten(), 
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(51200, 256),
    nn.ReLU(),
    nn.BatchNorm1d(256),
    nn.Dropout(0.5),
    nn.Linear(256, 22),
    Reshape(-1,11,2),
    nn.Tanh()
)

In [11]:
mse_loss_flat = MSELossFlat()

learn = cnn_learner(
    data, 
    models.resnet34, 
    loss_func=mse_loss_flat,
    custom_head=head_reg
).to_fp16()

In [12]:
learn.path = Path('..')

In [13]:
# learn.lr_find()
# learn.recorder.plot()

In [14]:
# learn.freeze_to(-1)
# learn.fit_one_cycle(
#     10, 
#     slice(1e-2), 
#     callbacks=[
#         SaveModelCallback(learn, every='improvement', monitor='valid_loss', name='resnet34_frozen_to_-1'),
#         EarlyStoppingCallback(learn, monitor='valid_loss', min_delta=0.01, patience=10)
#     ]
# )

In [15]:
learn = learn.load('fastai_resnet34')
learn = learn.to_fp32()

In [16]:
# learn.freeze_to(-2)
# learn.fit_one_cycle(
#     10, 
#     slice(1e-3), 
#     callbacks=[
#         SaveModelCallback(learn, every='improvement', monitor='valid_loss', name='resnet34_frozen_to_-2'),
#         EarlyStoppingCallback(learn, monitor='valid_loss', min_delta=0.01, patience=10)
#     ]
# )

In [17]:
# learn.show_results(rows=10)

## Deploy to AWS Lambda

In [18]:
%%writefile pet_regression.py
from bentoml import BentoService, api, env, artifacts 
from bentoml.artifact import FastaiModelArtifact
from bentoml.handlers import FastaiImageHandler

@env(pip_dependencies=['fastai', 'henrys_model_utils'])
@artifacts([FastaiModelArtifact('pet_regressor')])
class PetRegression(BentoService):
    
    @api(FastaiImageHandler)
    def predict(self, image):
        result = self.artifacts.pet_regressor.predict(image)
        return str(result)

Overwriting pet_regression.py


In [20]:
# 1) import the custom BentoService defined above
from pet_regression import PetRegression

# 2) `pack` it with required artifacts
service = PetRegression.pack(pet_regressor=learn)

# 3) save your BentoSerivce
saved_path = service.save()

[2019-11-12 22:56:44,338] INFO - Successfully saved Bento 'PetRegression:20191112225617_DA68CF' to path: /Users/HenryDashwood/bentoml/repository/PetRegression/20191112225617_DA68CF


In [21]:
from bentoml import load

service = load(saved_path)

print(service.predict(data.get(0)))

[2019-11-12 22:56:54,558] WARNING - Module `pet_regression` already loaded, using existing imported module.
(ImagePoints (320, 320), tensor([[-0.5638, -0.4552],
        [-0.8983, -0.4173],
        [-0.8985, -0.0678],
        [-0.9187,  0.2398],
        [-0.6159,  0.3104],
        [-0.2388, -0.0278],
        [-0.6171, -0.2045],
        [-0.6355,  0.1022],
        [-0.3086, -0.1297],
        [-0.3481, -0.0332],
        [-0.3190,  0.0545]]), tensor([[-0.5638, -0.4552],
        [-0.8983, -0.4173],
        [-0.8985, -0.0678],
        [-0.9187,  0.2398],
        [-0.6159,  0.3104],
        [-0.2388, -0.0278],
        [-0.6171, -0.2045],
        [-0.6355,  0.1022],
        [-0.3086, -0.1297],
        [-0.3481, -0.0332],
        [-0.3190,  0.0545]]))


In [22]:
!pip install --upgrade {saved_path}

Processing /Users/HenryDashwood/bentoml/repository/PetRegression/20191112225617_DA68CF
  Created wheel for PetRegression: filename=PetRegression-20191112225617_DA68CF-cp37-none-any.whl size=81256830 sha256=fa41be537f287ac158f2fc304c48c6ce0fb12bc9ea9e6a0e70aef36629877677
  Stored in directory: /private/var/folders/yl/c5z1v7897q3g9ywlj63pm0840000gn/T/pip-ephem-wheel-cache-i6v3y0yn/wheels/ce/e9/16/db29e8f3810731edafa65c11b94103c0cdf78f14ec0df295e3
Successfully built PetRegression


In [23]:
!PetRegression predict --input=../data/resized_images/Abyssinian_1.jpg

(ImagePoints (320, 320), tensor([[-0.3867,  0.0846],
        [-0.6847,  0.0874],
        [-0.5517,  0.2640],
        [-0.6337,  0.4377],
        [-0.3433,  0.4129],
        [-0.1768,  0.2797],
        [-0.3842,  0.1987],
        [-0.3603,  0.3473],
        [-0.2144,  0.2346],
        [-0.2251,  0.2833],
        [-0.2118,  0.3196]]), tensor([[-0.3867,  0.0846],
        [-0.6847,  0.0874],
        [-0.5517,  0.2640],
        [-0.6337,  0.4377],
        [-0.3433,  0.4129],
        [-0.1768,  0.2797],
        [-0.3842,  0.1987],
        [-0.3603,  0.3473],
        [-0.2144,  0.2346],
        [-0.2251,  0.2833],
        [-0.2118,  0.3196]]))


In [24]:
# !bentoml serve {saved_path}

In [25]:
bento_tag = '{name}:{version}'.format(name=service.name, version=service.version)
print(bento_tag)

PetRegression:20191112225617_DA68CF


In [30]:
!bentoml --verbose deployments delete facial-features-detector --force

[2019-11-13 07:43:25,306] DEBUG - Using BentoML with local Yatai server
[2019-11-13 07:43:25,942] DEBUG - Upgrading tables to the latest revision
Failed to delete deployment facial-features-detector. code: NOT_FOUND, message: Deployment "facial-features-detector" in namespace "default" not found


In [29]:
!bentoml --verbose deployment create facial-features-detector --bento {bento_tag} --platform aws-lambda --region us-east-1

[2019-11-12 23:08:24,143] DEBUG - Using BentoML with local Yatai server
[2019-11-12 23:08:24,391] DEBUG - Upgrading tables to the latest revision
[2019-11-12 23:08:24,793] DEBUG - Created temporary directory: /private/var/folders/yl/c5z1v7897q3g9ywlj63pm0840000gn/T/bentoml-temp-eqddothq
[2019-11-12 23:09:53,867] DEBUG - sls cmd output: Serverless: Generating boilerplate...
 _______                             __
|   _   .-----.----.--.--.-----.----|  .-----.-----.-----.
|   |___|  -__|   _|  |  |  -__|   _|  |  -__|__ --|__ --|
|____   |_____|__|  \___/|_____|__| |__|_____|_____|_____|
|   |   |             The Serverless Application Framework
|       |                           serverless.com, v1.53.0
 -------'

Serverless: Successfully generated boilerplate for template: "aws-python3"

[2019-11-12 23:09:57,367] INFO - Installing additional packages: serverless-python-requirements
[2019-11-12 23:10:15,064] DEBUG - sls cmd output: Serverless: Creating an empty package.json file in your

In [28]:
# !bentoml deployment list